In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

In [59]:
dfTrain = pd.read_csv('../../../DataSet/trip_train.csv', 
                      parse_dates=['start_date', 'end_date'], 
                      infer_datetime_format=True)

dfTest = pd.read_csv('../../../DataSet/trip_test.csv', 
                     parse_dates=['start_date', 'end_date'], 
                     infer_datetime_format=True)

dfWeather = pd.read_csv('../../../DataSet/weather.csv', parse_dates=['date'], infer_datetime_format=True)

dfStation = pd.read_csv('../../../DataSet/station.csv', parse_dates=['installation_date'], infer_datetime_format=True)

In [68]:
def getEventCategoryName(event):
    if pd.isnull(event):
        return 'Clear'
    capitalizedEvent = event.capitalize()
    if capitalizedEvent == 'Fog':
        return 'Clear'
    if capitalizedEvent == 'Fog-rain':
        return 'Rain'
    return capitalizedEvent


dfPrecipitations = dfWeather[['date', 'zip_code', 'events', 'precipitation_inches']]
dfPrecipitations.rename(columns={'date':'start_date'}, inplace=True)

dfPrecipitations.events = dfPrecipitations.events.apply(getEventCategoryName)
dfPrecipitations[dfPrecipitations.events == 'Clear'].precipitation_inches.unique()

array(['0', 'T', '0.01', '0.08', '0.05', '0.03', nan, '0.02'], dtype=object)

Como se puede ver, en los que quedan categorizados como 'Clear' las precipitaciones son muy bajas y, en su gran mayoría, el valor es 0. Por lo que tomo como correcta la categorización.

In [69]:
events = dfPrecipitations.events.unique()
print events
dfPrecipitations.events = dfPrecipitations.events.astype('category', categories=events).cat.codes
print dfPrecipitations.events.unique()

['Clear' 'Rain' 'Rain-thunderstorm']
[0 1 2]


In [21]:
''' Utilizada como parámetro para función DataFrame.apply
    Para cada row, devuelve el zip code correspondiente a la ciudad.'''
def cityNameToZipCode(row) :

    if row.city == 'San Francisco' :
        return 94107

    if row.city == 'Redwood City' :
        return 94063

    if row.city == 'Palo Alto' :
        return 94301

    if row.city == 'Mountain View' :
        return 94041

    if row.city == 'San Jose' :
        return 95113

dfStation.rename(columns={'id':'start_station_id'}, inplace=True)
#Agrego columna de events a dfTrain
dfTrain = dfTrain.merge(dfStation[['start_station_id', 'city']], on=['start_station_id'])
dfTrain.zip_code = dfTrain.apply(cityNameToZipCode, axis=1)
dfTrain.start_date = pd.to_datetime(dfTrain.start_date.dt.date)
dfTrain = dfTrain.merge(dfPrecipitations, on=['start_date', 'zip_code'])
#Agrego columna de events a dfTest
dfTest = dfTest.merge(dfStation[['start_station_id', 'city']], on=['start_station_id'])
dfTest.zip_code = dfTest.apply(cityNameToZipCode, axis=1)
dfTest.start_date = pd.to_datetime(dfTest.start_date.dt.date)
dfTest = dfTest.merge(dfPrecipitations, on=['start_date', 'zip_code'])

In [22]:
# Convierto a los SUSCRIBER en un 0
# Convierto a los CUSTOMER en un 1
subscriptionTypes = dfTrain.subscription_type.unique()
print subscriptionTypes

# Reemplazo por 0 y 1
dfTrain.subscription_type = dfTrain.subscription_type.astype('category', categories=subscriptionTypes).cat.codes
dfTest.subscription_type = dfTest.subscription_type.astype('category', categories=subscriptionTypes).cat.codes

['Subscriber' 'Customer']


In [23]:
# GENERO TARGET, TRAIN, TEST Y TESTIDS (testids para el output)
target = dfTrain.duration
testIds = dfTest['id']


trainDateData = {'start_month':dfTrain.start_date.dt.month,
                 'start_dayOfWeek':dfTrain.start_date.dt.dayofweek, 
                 'start_hourOfDay':dfTrain.start_date.dt.hour}

testDateData = {'start_month':dfTest.start_date.dt.month,
                'start_dayOfWeek':dfTest.start_date.dt.dayofweek, 
                'start_hourOfDay':dfTest.start_date.dt.hour}

# Agrego columnas con la informacion de fechas a los dataframes
dfTrain = dfTrain[['start_station_id', 'subscription_type', 'events']].join(pd.DataFrame(trainDateData), how='outer')
dfTest = dfTest[['start_station_id', 'subscription_type', 'events']].join(pd.DataFrame(testDateData), how='outer')



In [24]:
'''
rf = RandomForestRegressor(n_estimators=3, n_jobs=-1)

print("Volcando puntos...")
rf.fit(dfTrain, target)

print("Prediciendo...")
predictions = rf.predict(dfTest)

'''

'\nrf = RandomForestRegressor(n_estimators=3, n_jobs=-1)\n\nprint("Volcando puntos...")\nrf.fit(dfTrain, target)\n\nprint("Prediciendo...")\npredictions = rf.predict(dfTest)\n\n'

In [25]:
#zip(predictions, testIds)